In [22]:
import warnings
warnings.filterwarnings("ignore")


In [23]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [24]:
from datetime import datetime
from src.data_split import  train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32226, 674)
y_train.shape=(32226,)
X_test.shape=(56068, 674)
y_test.shape=(56068,)


In [25]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
         
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [34]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-03-12 19:44:28,358] A new study created in memory with name: no-name-8a229a85-cce2-437f-98b8-6307867cb7d0


[I 2024-03-12 19:44:47,146] Trial 0 finished with value: 3.4736819891178263 and parameters: {'num_leaves': 28, 'feature_fraction': 0.45552675319908453, 'bagging_fraction': 0.23374529056265053, 'min_child_samples': 39}. Best is trial 0 with value: 3.4736819891178263.
[I 2024-03-12 19:45:05,661] Trial 1 finished with value: 3.3010282141928085 and parameters: {'num_leaves': 36, 'feature_fraction': 0.4179622500924738, 'bagging_fraction': 0.9301628983838077, 'min_child_samples': 14}. Best is trial 1 with value: 3.3010282141928085.
[I 2024-03-12 19:46:13,747] Trial 2 finished with value: 3.349739343896582 and parameters: {'num_leaves': 225, 'feature_fraction': 0.9231368779848472, 'bagging_fraction': 0.745998914919199, 'min_child_samples': 27}. Best is trial 1 with value: 3.3010282141928085.
[I 2024-03-12 19:47:04,046] Trial 3 finished with value: 3.1058449884445096 and parameters: {'num_leaves': 194, 'feature_fraction': 0.7387593145376956, 'bagging_fraction': 0.623082871284614, 'min_child_sa

In [35]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 194, 'feature_fraction': 0.7387593145376956, 'bagging_fraction': 0.623082871284614, 'min_child_samples': 17}


In [36]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.7387593145376956, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7387593145376956
[LightGBM] [Warning] bagging_fraction is set=0.623082871284614, subsample=1.0 will be ignored. Current value: bagging_fraction=0.623082871284614
[LightGBM] [Warning] feature_fraction is set=0.7387593145376956, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7387593145376956
[LightGBM] [Warning] bagging_fraction is set=0.623082871284614, subsample=1.0 will be ignored. Current value: bagging_fraction=0.623082871284614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.727835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 154527
[LightGBM] [Info] Number of data points in the train set: 32226, number of used features: 675
[LightGBM] [Info] Start training from score 11.703562


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000252CE8D4040>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.623082871284614,
                               feature_fraction=0.7387593145376956,
                               min_child_samples=17, num_leaves=194))])

In [39]:
predictions = pipeline.predict(X_test)
from sklearn.metrics import  mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.7387593145376956, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7387593145376956
[LightGBM] [Warning] bagging_fraction is set=0.623082871284614, subsample=1.0 will be ignored. Current value: bagging_fraction=0.623082871284614
test_mae=2.5934


In [40]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=90,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [33]:
plot_one_sample(
    example_id=135,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)